In [1043]:
setwd("/projects/PPC/analysis/ppc_eqtls")

source("scripts/packages.R"  )
source("scripts/functions.R" )
source("scripts/input_data.R")

library(coloc)

set.seed(5366)

# Import 1000 Genomes rsids

In [2]:
rsid_list = list()

for (i in c(1:22))
{
    message(i, appendLF = F)
    this = paste("VAR", readLines(paste("/frazer01/home/jennifer/references/1kg/vcf", paste(paste0("chr", i), "snp", sep = "."), sep = "/")), sep = "_")
    this = data.frame(snp = this)
    this[,c("var", "chr", "pos", "ref", "alt")] = str_split_fixed(this$snp, "_", 5)
    this$chr_pos = paste(this$chr, this$pos, sep = "_")
    rsid_list[[paste0("chr", i)]] = this
}

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


# Get causal SNPs for each eQTL

In [4]:
load("manuscript/robj/egene_list.robj", verbose = T)
names(egene_list)

load("manuscript/robj/gt1_list.robj", verbose = T)
names(gt1_list)


Loading objects:
  egene_list


[1] "gene"    "isoform"

Loading objects:
  gt1_list


[1] "gene"    "isoform"

In [32]:
# consolidate all eqtls
all_qtls = rbindlist(lapply(names(egene_list), function(pheno) 
{
    rbindlist(lapply(names(egene_list[[pheno]]), function(tiss)
    {
        egene_list[[pheno]][[tiss]] %>% mutate(qtl_id = gsub(" ", "_", qtl_id)) %>% select(qtl_id) %>% mutate(tissue = tiss, analysis = pheno)
    }))
})) 

# rename tissues to be easier
all_qtls[all_qtls$tissue == "iPSC-PPC",]$tissue = "ipsc-ppc"
all_qtls[all_qtls$tissue == "Islets",]$tissue = "islet"
all_qtls[all_qtls$tissue == "Pancreas",]$tissue = "pancreas"

In [26]:
# consolidate all fine-mapped snps with pp > 1%
snps_gt1 = rbindlist(lapply(names(gt1_list), function(pheno)
{
    rbindlist(lapply(names(gt1_list[[pheno]]), function(tiss)
    {
        gt1_list[[pheno]][[tiss]] %>% select(SNP.PP, snp, qtl_id) %>% mutate(tissue = tiss, analysis = pheno) %>% filter(SNP.PP >= 0.01)
    }))
})) %>% mutate(snp = gsub("VAR_", "", snp), chrom = unlist(lapply(snp, function(x) { unlist(strsplit(x, "_"))[1] })))

# get chrom and pos
snps_gt1$chrom = as.numeric(snps_gt1$chrom)
snps_gt1$pos = as.numeric(unlist(lapply(snps_gt1$snp, function(x) { unlist(strsplit(x, "_"))[2] })))

# rename tissue
snps_gt1[snps_gt1$tissue == "iPSC-PPC",]$tissue = "ipsc-ppc"
snps_gt1[snps_gt1$tissue == "Islets",]$tissue = "islet"
snps_gt1[snps_gt1$tissue == "Pancreas",]$tissue = "pancreas"

# get discovery order
snps_gt1$type = unlist(lapply(snps_gt1$qtl_id, function(x) { unlist(strsplit(x, "_"))[1] }))
snps_gt1[snps_gt1$type %in% c("Islets", "Pancreas"),]$type = 0

# get transcript_id
snps_gt1$transcript_id = unlist(lapply(snps_gt1$qtl_id, function(x) { unlist(strsplit(x, "_"))[2] }))

# snps for only eqtls that passed criteria
snps_gt1 = snps_gt1 %>% filter(qtl_id %in% all_qtls$qtl_id)

head(snps_gt1)

SNP.PP,snp,qtl_id,tissue,analysis,chrom,pos,type,transcript_id
<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
0.01067617,20_50005043_G_GA,ipsc-ppc_0_ENSG00000000419,ipsc-ppc,gene,20,50005043,0,ENSG00000000419
0.03152537,6_46050368_G_C,ipsc-ppc_0_ENSG00000001561,ipsc-ppc,gene,6,46050368,0,ENSG00000001561
0.01674834,6_46050598_T_A,ipsc-ppc_0_ENSG00000001561,ipsc-ppc,gene,6,46050598,0,ENSG00000001561
0.01674834,6_46058361_C_T,ipsc-ppc_0_ENSG00000001561,ipsc-ppc,gene,6,46058361,0,ENSG00000001561
0.02301381,6_46060370_TTG_T,ipsc-ppc_0_ENSG00000001561,ipsc-ppc,gene,6,46060370,0,ENSG00000001561
0.03152537,6_46064645_A_G,ipsc-ppc_0_ENSG00000001561,ipsc-ppc,gene,6,46064645,0,ENSG00000001561


In [40]:
table(snps_gt1$analysis, snps_gt1$tissue)

         
          ipsc-ppc  islet pancreas
  gene       57771  70141   116309
  isoform    54787 106363    29453

In [41]:
# check that all eQTLs are present
eqtl_annot = fread("reviews/tables/Table_S11_eQTL_Annotations.txt", data.table = F)
eqtl_annot %>% filter(!eqtl_id %in% snps_gt1$qtl_id)

Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


module_id,eqtl_id,transcript_id,gene_id,gene_name,developmental_stage,tissue,eqtl_phenotype,eqtl_type,expressed_ipsc_ppc,⋯,LD_ipsc_ppc,LD_islets,LD_pancreas,LD_islets_conditional,LD_pancreas_conditional,islet_egene_overlap,pancreas_egene_overlap,module_pass,category_annotation,notes
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<chr>


# Get adult conditional lead snps

In [42]:
load("input/eqtl/pancreas_islets_conditionals.robj", verbose = T)
conds = fread("input/eqtl/pancreas_islets_conditionals.txt", data.table = F)

Loading objects:
  cond_list


In [58]:
# reformat, arbitrarily assign SNP.PP 1% 
conds2 = conds %>% mutate(snp = hg19_snp, qtl_id = paste(tissue, rank, phenotype_id, sep = "_")) %>% 
    mutate(SNP.PP = 0.01, type = rank) %>% dplyr::rename(chrom = chr) %>% select(SNP.PP, type, snp, qtl_id, tissue, analysis, chrom, pos)

# rename tissues
conds2[conds2$tissue == "Pancreas",]$tissue = "pancreas"
conds2[conds2$tissue == "Islets",]$tissue = "islet"

# rename qtl_ids
conds2$qtl_id = gsub("Pancreas", "pancreas", conds2$qtl_id)
conds2$qtl_id = gsub("Islets", "islet", conds2$qtl_id)

# add analysis and transcript_id column
conds2$analysis = ifelse(conds$analysis %like% "gene", "gene", "isoform")
conds2$transcript_id = unlist(lapply(conds2$qtl_id, function(x) { x = unlist(strsplit(x, "_")); paste(x[3:length(x)], collapse = "_") }))

In [59]:
# make sure the columns are the same with snps_gt1
setdiff(colnames(conds2), colnames(snps_gt1))
setdiff(colnames(snps_gt1), colnames(conds2))

character(0)

character(0)

In [60]:
head(conds2,2)
head(snps_gt1,2)

,SNP.PP,type,snp,qtl_id,tissue,analysis,chrom,pos,transcript_id
,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>
1,0.01,1,1_100650555_T_C,pancreas_1_ENSG00000122477,pancreas,gene,1,100650555,ENSG00000122477
2,0.01,1,1_100717447_G_T,pancreas_1_ENSG00000137996,pancreas,gene,1,100717447,ENSG00000137996


SNP.PP,snp,qtl_id,tissue,analysis,chrom,pos,type,transcript_id
<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
0.01067617,20_50005043_G_GA,ipsc-ppc_0_ENSG00000000419,ipsc-ppc,gene,20,50005043,0,ENSG00000000419
0.03152537,6_46050368_G_C,ipsc-ppc_0_ENSG00000001561,ipsc-ppc,gene,6,46050368,0,ENSG00000001561


In [63]:
snps_gt1 = rbind(snps_gt1, conds2) %>% distinct()

# Check that all eQTLs are present

In [70]:
# get all adult conditional eqtls
cond_list[["panc_eqtl"]]  = cond_list[["panc_eqtl" ]] %>% mutate(qtl_id = paste("pancreas", rank, simplify_id(phenotype_id), sep = "_"))
cond_list[["panc_sqtl"]]  = cond_list[["panc_sqtl" ]] %>% mutate(qtl_id = paste("pancreas", rank, phenotype_id             , sep = "_"))
cond_list[["islet_eqtl"]] = cond_list[["islet_eqtl"]] %>% mutate(qtl_id = paste("islet"   , rank, simplify_id(phenotype_id), sep = "_"))
cond_list[["islet_sqtl"]] = cond_list[["islet_sqtl"]] %>% mutate(qtl_id = paste("islet"   , rank, phenotype_id             , sep = "_"))

In [83]:
head(cond_list[["islet_eqtl"]])

,hg19_snp,rank,phenotype_id,chr,tissue,analysis,qtl_id
,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,3_53909731_C_A,1,ENSG00000113812.9,3,islets,gene,islet_1_ENSG00000113812
2,3_56734630_C_T,1,ENSG00000187672.8,3,islets,gene,islet_1_ENSG00000187672
3,3_56726105_A_C,1,ENSG00000180376.12,3,islets,gene,islet_1_ENSG00000180376
4,3_56793722_A_G,1,ENSG00000163947.7,3,islets,gene,islet_1_ENSG00000163947
5,3_58280061_T_C,1,ENSG00000163686.9,3,islets,gene,islet_1_ENSG00000163686
6,3_58277536_A_G,1,ENSG00000168297.11,3,islets,gene,islet_1_ENSG00000168297


In [85]:
# check that there are no sex chromosomes - we are only considering autosomes
cond_list[["panc_eqtl" ]] %>% filter(!qtl_id %in% snps_gt1$qtl_id) %>% filter(chr != "X")
cond_list[["panc_sqtl" ]] %>% filter(!qtl_id %in% snps_gt1$qtl_id) %>% filter(chr != "X")
cond_list[["islet_eqtl"]] %>% filter(!qtl_id %in% snps_gt1$qtl_id) %>% filter(chr != "23")
cond_list[["islet_sqtl"]] %>% filter(!qtl_id %in% snps_gt1$qtl_id) %>% filter(chr != "23")

hg19_snp,rank,phenotype_id,chr,tissue,analysis,qtl_id
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>


hg19_snp,rank,phenotype_id,chr,tissue,analysis,qtl_id
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>


hg19_snp,rank,phenotype_id,chr,tissue,analysis,qtl_id
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>


hg19_snp,rank,phenotype_id,chr,tissue,analysis,qtl_id
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>


In [86]:
# check again that all ppc qtls and primary adults are present 
eqtl_annot = fread("reviews/tables/Table_S11_eQTL_Annotations.txt", data.table = F)
eqtl_annot %>% filter(!eqtl_id %in% snps_gt1$qtl_id)

Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


module_id,eqtl_id,transcript_id,gene_id,gene_name,developmental_stage,tissue,eqtl_phenotype,eqtl_type,expressed_ipsc_ppc,⋯,LD_ipsc_ppc,LD_islets,LD_pancreas,LD_islets_conditional,LD_pancreas_conditional,islet_egene_overlap,pancreas_egene_overlap,module_pass,category_annotation,notes
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<chr>


# Add rsids

In [87]:
# reformat snp id as "VAR_[chr]_[pos]_[ref]_[alt]"
snps_gt1$snp = paste("VAR", snps_gt1$snp, sep = "_")

# annotate each snp TRUE/FALSE if they are genotyped in 1000 Genomes
snps_gt1 = as.data.frame(rbindlist(lapply(c(1:22), function(x)
{
    message(x, appendLF = F)
    snps_gt1 %>% filter(chrom == x) %>% mutate(in_1kg = ifelse(snp %in% rsid_list[[paste0("chr", x)]]$snp, T, F))
})))

# reformat snp id to remove "VAR_" in front
snps_gt1$snp = gsub("VAR_", "", snps_gt1$snp)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


# Choose SNPs to test LD with

In [102]:
# prioritize snps in 1000 Genomes, select the one with max SNP.PP, but if multiple SNPs have max SNP.PP, choose randomly
top = snps_gt1 %>% filter(in_1kg == T) %>% group_by(qtl_id) %>% filter(SNP.PP == max(SNP.PP)) %>% filter(duplicated(qtl_id) == F)

# for qtls with no snps in 1000 Genomes, select the snp with highest SNP.PP, 
top = rbind(top, snps_gt1 %>% filter(!qtl_id %in% top$qtl_id) %>% group_by(SNP.PP) %>% filter(SNP.PP == max(SNP.PP)) %>% filter(duplicated(qtl_id) == F))
top = top %>% filter(duplicated(qtl_id) == F)

In [106]:
# check that all eqtls have a testing snp
eqtl_annot = fread("reviews/tables/Table_S11_eQTL_Annotations.txt", data.table = F)
eqtl_annot %>% filter(!eqtl_id %in% top$qtl_id)

cond_list[["panc_eqtl" ]] %>% filter(!qtl_id %in% top$qtl_id) %>% filter(chr != "X")
cond_list[["panc_sqtl" ]] %>% filter(!qtl_id %in% top$qtl_id) %>% filter(chr != "X")
cond_list[["islet_eqtl"]] %>% filter(!qtl_id %in% top$qtl_id) %>% filter(chr != "23")
cond_list[["islet_sqtl"]] %>% filter(!qtl_id %in% top$qtl_id) %>% filter(chr != "23")

Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


module_id,eqtl_id,transcript_id,gene_id,gene_name,developmental_stage,tissue,eqtl_phenotype,eqtl_type,expressed_ipsc_ppc,⋯,LD_ipsc_ppc,LD_islets,LD_pancreas,LD_islets_conditional,LD_pancreas_conditional,islet_egene_overlap,pancreas_egene_overlap,module_pass,category_annotation,notes
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<chr>


hg19_snp,rank,phenotype_id,chr,tissue,analysis,qtl_id
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>


hg19_snp,rank,phenotype_id,chr,tissue,analysis,qtl_id
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>


hg19_snp,rank,phenotype_id,chr,tissue,analysis,qtl_id
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>


hg19_snp,rank,phenotype_id,chr,tissue,analysis,qtl_id
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>


# Save the testing snps

In [108]:
# number of snps to test per chromosome
table(top$chrom)


   1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16 
5306 3893 3202 2133 2703 3517 3358 1857 2293 2387 3208 2678  967 1868 1953 2717 
  17   18   19   20   21   22 
3247  766 4043 1388  800 1712 

In [110]:
# Remove "VAR_" from snp id to match with VCF snp ids
top$snp = gsub("VAR_", "", top$snp)

# un-factorise dataframe
top = as.data.frame(top %>% ungroup())

In [224]:
# save
fwrite(top, "reviews/LD/r2/top.txt", row.names = F, sep = "\t")

In [109]:
head(top,2)

SNP.PP,snp,qtl_id,tissue,analysis,chrom,pos,type,transcript_id,in_1kg
<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<lgl>
0.04718508,1_113061959_G_C,ipsc-ppc_0_ENSG00000007341,ipsc-ppc,gene,1,113061959,0,ENSG00000007341,TRUE
0.10571673,1_1612540_C_T,ipsc-ppc_0_ENSG00000008128,ipsc-ppc,gene,1,1612540,0,ENSG00000008128,TRUE


# Run plink r2

In [114]:
# run plink on qsub

for (c in c(1:22))
{
    outfile = paste("reviews/LD/r2/out", paste0("chr", c), sep = "/") # after reviews
    snpfile = paste("reviews/LD/r2/input", paste0("chr", c, ".txt"), sep = "/") # after reviews
    logout = paste("reviews/LD/r2/logs", paste(paste0("chr", c), "out", sep = "."), sep = "/")
    logerr = paste("reviews/LD/r2/logs", paste(paste0("chr", c), "err", sep = "."), sep = "/")
    
    writeLines(unique(top[top$chrom == c,]$snp), snpfile, sep = "\n")
    
    cmd = paste("plink", "--memory 15000 --threads 4", "--vcf", paste0("~/references/1kg/vcf/", paste0("chr", c), ".vcf.gz"), 
          "--extract", snpfile, "--keep-allele-order", "--r2", "square", "--make-bed", "--out", outfile)

    qsub_cmd = paste("echo", paste0("\"", cmd, "\"" ), "|", "qsub -N", paste0("chr", c),
                     "-pe smp 4", "-V -cwd", 
                     "-o", log_out,
                     "-e", log_err)
    message(qsub_cmd)
    system(qsub_cmd)
}


echo "plink --memory 15000 --threads 4 --vcf ~/references/1kg/vcf/chr1.vcf.gz --extract reviews/LD/r2/input/chr1.txt --keep-allele-order --r2 square --make-bed --out reviews/LD/r2/out/chr1" | qsub -N chr1 -pe smp 4 -V -cwd -o reviews/LD/r2/logs/chr1.out -e reviews/LD/r2/logs/chr1.err

echo "plink --memory 15000 --threads 4 --vcf ~/references/1kg/vcf/chr2.vcf.gz --extract reviews/LD/r2/input/chr2.txt --keep-allele-order --r2 square --make-bed --out reviews/LD/r2/out/chr2" | qsub -N chr2 -pe smp 4 -V -cwd -o reviews/LD/r2/logs/chr2.out -e reviews/LD/r2/logs/chr2.err

echo "plink --memory 15000 --threads 4 --vcf ~/references/1kg/vcf/chr3.vcf.gz --extract reviews/LD/r2/input/chr3.txt --keep-allele-order --r2 square --make-bed --out reviews/LD/r2/out/chr3" | qsub -N chr3 -pe smp 4 -V -cwd -o reviews/LD/r2/logs/chr3.out -e reviews/LD/r2/logs/chr3.err

echo "plink --memory 15000 --threads 4 --vcf ~/references/1kg/vcf/chr4.vcf.gz --extract reviews/LD/r2/input/chr4.txt --keep-allele-order --r2 

# Process results

In [423]:
top = fread("reviews/LD/r2/top.txt", data.table = F)

In [829]:
eqtl_annot = fread("reviews/tables/Table_S11_eQTL_Annotations.txt", data.table = F) 

# merge testing snp with their eqtls
eqtl_annot = merge(eqtl_annot, top %>% select(chrom, qtl_id) %>% dplyr::rename(eqtl_id = qtl_id), all.x = T)

# add module_id column - if singleton, use the eqtl_id, if not, use the module_id
eqtl_annot$module_id = ifelse(eqtl_annot$eqtl_type == "singleton", eqtl_annot$eqtl_id, eqtl_annot$module_id)

# annotation whether gene or isoform (i.e., alternative splicing)
eqtl_annot$analysis = ifelse(eqtl_annot$eqtl_phenotype %like% "gene", "gene", "isoform")

# give taskid number for qsub purposes
eqtl_annot$taskid = c(1:nrow(eqtl_annot))


    - run on qsub projects/PPC/analysis/ppc_eqtls/reviews/process_LD_r2.R

#### test script on one eqtl

In [1057]:
row = 1041

In [1069]:
out = qtl2class[row,]

qtlid      = qtl2class[row,]$eqtl_id
tiss       = qtl2class[row,]$tissue
mod_id     = qtl2class[row,]$module_id
phen       = qtl2class[row,]$analysis
snp        = top[top$qtl_id == qtlid,]$snp
position   = as.numeric(unlist(strsplit(snp, "_"))[2])
eqtltype   = qtl2class[row,]$eqtl_type
transcript = unlist(strsplit(qtlid, "_"))
transcript = paste(transcript[3:length(transcript)], collapse = "_")

message(paste("qtl_id:"    , qtlid))
message(paste("qtltype:"   , eqtltype))
message(paste("transcript:", transcript))
message(paste("phenotype:" , phen))
message(paste("snp:"       , snp))

chr = unlist(strsplit(snp, "_"))[1]
message(paste("chromosome:", chr))

# Get eqtls that won't be tested, i.e. other conditionals of the same transcript or other eqtls in the same module
qtls1 = qtl2class[qtl2class$analysis == phen & 
                  qtl2class$chrom == chr & 
                  qtl2class$tissue == tiss & 
                  (qtl2class$eqtl_id %like% transcript | qtl2class$module_id == mod_id),]$eqtl_id

qtls = unique(c(qtls1, qtl2class[qtl2class$analysis == phen & qtl2class$module_id == mod_id,]$eqtl_id))
message(paste("Do not test these qtls:", paste(qtls, collapse = "\n"), sep = "\n"))

message("Testing these QTLs for LD within 500 kb: ================")
to_test = top %>% filter(chrom == chr & analysis == phen & !qtl_id %in% qtls & pos >= (position - 500e3) & pos <= (position + 500e3))

if (tiss %in% c("pancreas", "islet"))
{
    remove = to_test %>% filter(qtl_id %like% transcript & tissue == tiss)
    print(paste("Removed:", paste(remove$qtl_id, collapse = ", ")))
    to_test = to_test %>% filter(!qtl_id %in% remove$qtl_id)
}

message("Opening LD")
bim = fread(paste("reviews/LD/r2/out", paste0("chr", chr, ".bim"), sep = "/"), data.table = F)
ld = fread(paste("reviews/LD/r2/out", paste0("chr", chr, ".ld"), sep = "/"), data.table = F)
rownames(ld) = bim$V2
colnames(ld) = bim$V2

ld = suppressWarnings(suppressMessages(melt(as.matrix(ld))))

ld2 = ld %>% filter(Var1 == snp & Var2 %in% to_test$snp)
print(summary(ld2$value))
to_test$r2 = as.numeric(suppressMessages(mapvalues(to_test$snp, from = ld2$Var2, to = ld2$value)))
to_test$in_r2 = ifelse(to_test$snp %in% ld2$Var2, ifelse(to_test$snp %in% ld2[ld2$value >= 0.2,]$Var2, T, F), "not_tested")
to_test$in_window = abs(to_test$pos - position)
to_test$in_ld = ifelse(to_test$in_r2 == T, T, ifelse(to_test$in_r2 == "not_tested" & to_test$in_window <= 500e3, T, F))
table(to_test$tissue, to_test$in_ld)

message("Annotated for LD")
# to_test %>% arrange(in_r2)
# to_test %>% arrange(desc(SNP.PP))
to_test %>% arrange(desc(in_window))

out$ld_ipsc_ppc = ifelse(nrow(to_test[to_test$tissue == "ipsc-ppc" & to_test$in_ld == T,]) > 0, T, F)
out$ld_islet = ifelse(nrow(to_test[to_test$tissue == "islet" & to_test$in_ld == T,]) > 0, T, F)
out$ld_pancreas = ifelse(nrow(to_test[to_test$tissue == "pancreas" & to_test$in_ld == T,]) > 0, T, F)

return(out)

qtl_id: ipsc-ppc_0_ENSG00000128311

qtltype: singleton

transcript: ENSG00000128311

phenotype: gene

snp: 22_37405408_G_T

chromosome: 22

Do not test these qtls:
ipsc-ppc_0_ENSG00000128311

Testing these QTLs: ================

Opening LD



     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
3.930e-06 4.542e-04 2.491e-03 1.091e-02 7.705e-03 9.179e-02 


          
           FALSE
  ipsc-ppc     3
  islet        8
  pancreas    18

Annotated for LD



SNP.PP,snp,qtl_id,tissue,analysis,chrom,pos,type,transcript_id,in_1kg,r2,in_r2,in_window,in_ld
<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<lgl>,<dbl>,<lgl>,<dbl>,<lgl>
0.07657956,22_36905776_G_A,pancreas_0_ENSG00000100350,pancreas,gene,22,36905776,0,ENSG00000100350,TRUE,5.23631e-02,FALSE,499632,FALSE
0.05335938,22_36914347_C_G,islet_0_ENSG00000100348,islet,gene,22,36914347,0,ENSG00000100348,TRUE,3.93315e-06,FALSE,491061,FALSE
0.01000000,22_36914347_C_G,islet_1_ENSG00000100348,islet,gene,22,36914347,1,ENSG00000100348,TRUE,3.93315e-06,FALSE,491061,FALSE
0.01000000,22_37008928_A_G,pancreas_2_ENSG00000100360,pancreas,gene,22,37008928,2,ENSG00000100360,TRUE,7.11913e-06,FALSE,396480,FALSE
0.29685296,22_37798644_G_T,islet_0_ENSG00000166897,islet,gene,22,37798644,0,ENSG00000166897,TRUE,1.07757e-03,FALSE,393236,FALSE
0.01000000,22_37798644_G_T,islet_1_ENSG00000166897,islet,gene,22,37798644,1,ENSG00000166897,TRUE,1.07757e-03,FALSE,393236,FALSE
0.09109036,22_37797138_A_G,pancreas_0_ENSG00000166897,pancreas,gene,22,37797138,0,ENSG00000166897,TRUE,1.45221e-04,FALSE,391730,FALSE
0.01000000,22_37797138_A_G,pancreas_1_ENSG00000166897,pancreas,gene,22,37797138,1,ENSG00000166897,TRUE,1.45221e-04,FALSE,391730,FALSE
0.04098229,22_37727452_C_G,ipsc-ppc_0_ENSG00000189060,ipsc-ppc,gene,22,37727452,0,ENSG00000189060,TRUE,3.64086e-03,FALSE,322044,FALSE


,eqtl_id,module_id,transcript_id,gene_id,gene_name,developmental_stage,tissue,eqtl_phenotype,eqtl_type,expressed_ipsc_ppc,⋯,pancreas_egene_overlap,module_pass,category_annotation,notes,chrom,analysis,taskid,ld_ipsc_ppc,ld_islet,ld_pancreas
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,⋯,<chr>,<lgl>,<chr>,<chr>,<int>,<chr>,<int>,<lgl>,<lgl>,<lgl>
1041,ipsc-ppc_0_ENSG00000128311,ipsc-ppc_0_ENSG00000128311,ENSG00000128311,ENSG00000128311,TST,fetal-like,ipsc-ppc,gene_expression,singleton,TRUE,⋯,,FALSE,ambiguous,In LD with another eQTL,22,gene,1041,FALSE,FALSE,FALSE


# Get eQTL annotations before LD analysis

In [804]:
load("pipeline/3.4.coloc_adult/summary/panc_islets.coloc_list.pp0.8.all_panc.gt1.robj", verbose = T)

Loading objects:
  graphlist


In [805]:
before_ld = graphlist$qtl2class %>% select(eqtl_id, tissue, category_before_ld)

# rename eqtl
before_ld$eqtl_id = gsub(" ", "_", before_ld$eqtl_id)
before_ld$eqtl_id = ifelse(before_ld$tissue == "ipsc_ppc", paste0("ipsc-ppc_", before_ld$eqtl_id), before_ld$eqtl_id)
before_ld$eqtl_id = gsub("Pancreas", "pancreas_0", before_ld$eqtl_id)
before_ld$eqtl_id = gsub("Islets", "islet_0", before_ld$eqtl_id)

# rename tissue
before_ld$category_before_ld = gsub("endocrine", "islet", before_ld$category_before_ld)
before_ld$category_before_ld = gsub("exocrine", "pancreas", before_ld$category_before_ld)

# rename some annotations
before_ld$category_before_ld = ifelse(before_ld$category_before_ld == "ppc-unique", "ipsc_ppc-unique", before_ld$category_before_ld)
before_ld$category_before_ld = ifelse(before_ld$category_before_ld == "panc-unique", "pancreas-unique", before_ld$category_before_ld)
before_ld$category_before_ld = ifelse(before_ld$category_before_ld == "adult-unique", "adult-shared", before_ld$category_before_ld)

# eqtl annotations before applying LD filter
table(before_ld$category_before_ld)


head(before_ld)


      adult-shared        fetal-adult        fetal-islet     fetal-pancreas 
              3413               4072               1015               1657 
ipsc_ppc singleton    ipsc_ppc-unique    islet singleton       islet-unique 
              4528                815               4453                961 
pancreas singleton    pancreas-unique 
              5648                822 

,eqtl_id,tissue,category_before_ld
,<chr>,<chr>,<chr>
1,ipsc-ppc_0_ENSG00000000419,ipsc_ppc,ipsc_ppc singleton
2,ipsc-ppc_0_ENSG00000002726,ipsc_ppc,ipsc_ppc singleton
3,ipsc-ppc_0_ENSG00000003436,ipsc_ppc,ipsc_ppc singleton
4,ipsc-ppc_0_ENSG00000003987,ipsc_ppc,ipsc_ppc singleton
5,ipsc-ppc_0_ENSG00000005102,ipsc_ppc,ipsc_ppc singleton
6,ipsc-ppc_0_ENSG00000005194,ipsc_ppc,ipsc_ppc singleton


# Get processed LD results

    - This table tells for each eQTL, whether it's in LD with an iPSC-PPC eQTL, LD with an islet eQTL, and LD with a pancreas eQTL

In [1002]:
out = as.data.frame(rbindlist(lapply(c(1:22), function(taskid)
{
    file = paste("reviews/LD/r2/processed", paste0("chr", taskid, ".txt"), sep = "/")
    fread(file, data.table = F)
})))

# merge annotations before LD
out = merge(out, before_ld %>% select(-tissue), by = "eqtl_id", all.x = T)

In [1003]:
colnames(out)

[1] "eqtl_id"                 "module_id"              
 [3] "transcript_id"           "gene_id"                
 [5] "gene_name"               "developmental_stage"    
 [7] "tissue"                  "eqtl_phenotype"         
 [9] "eqtl_type"               "expressed_ipsc_ppc"     
[11] "expressed_endocrine"     "expressed_exocrine"     
[13] "LD_ipsc_ppc"             "LD_islets"              
[15] "LD_pancreas"             "LD_islets_conditional"  
[17] "LD_pancreas_conditional" "islet_egene_overlap"    
[19] "pancreas_egene_overlap"  "module_pass"            
[21] "category_annotation"     "notes"                  
[23] "chrom"                   "analysis"               
[25] "taskid"                  "ld_ipsc_ppc"            
[27] "ld_islet"                "ld_pancreas"            
[29] "category_before_ld"

In [1004]:
table(out$category_before_ld)


      adult-shared        fetal-adult        fetal-islet     fetal-pancreas 
              3413               4072               1015               1657 
ipsc_ppc singleton    ipsc_ppc-unique    islet singleton       islet-unique 
              4528                815               4453                961 
pancreas singleton    pancreas-unique 
              5648                822 

# Update annotations based on LD

In [1005]:
out$annotation = "ambiguous"
out$annotation = ifelse(out$category_before_ld == "adult-shared" &
                        out$ld_ipsc_ppc == F,
                        "adult-shared",
                        out$annotation)

out$annotation = ifelse(out$category_before_ld == "fetal-islet" &
                        out$ld_pancreas == F,
                        "fetal-islet",
                        out$annotation)

out$annotation = ifelse(out$category_before_ld == "fetal-pancreas" &
                        out$ld_islet == F,
                        "fetal-pancreas",
                        out$annotation)

out$annotation = ifelse(out$category_before_ld == "ipsc_ppc singleton" &
                        out$ld_islet == F &
                        out$ld_ipsc_ppc == F &
                        out$ld_pancreas == F,
                        "ipsc_ppc singleton",
                        out$annotation)

out$annotation = ifelse(out$category_before_ld == "ipsc_ppc-unique" &
                        out$ld_islet == F &
                        out$ld_pancreas == F,
                        "ipsc_ppc-unique",
                        out$annotation)

out$annotation = ifelse(out$category_before_ld == "islet singleton" &
                        out$ld_islet == F &
                        out$ld_ipsc_ppc == F &
                        out$ld_pancreas == F,
                        "islet singleton",
                        out$annotation)

out$annotation = ifelse(out$category_before_ld == "islet-unique" &
                        out$ld_ipsc_ppc == F &
                        out$ld_pancreas == F,
                        "islet-unique",
                        out$annotation)

out$annotation = ifelse(out$category_before_ld == "pancreas singleton" &
                        out$ld_ipsc_ppc == F &
                        out$ld_islet == F &
                        out$ld_pancreas == F,
                        "pancreas singleton",
                        out$annotation)

out$annotation = ifelse(out$category_before_ld == "pancreas-unique" &
                        out$ld_ipsc_ppc == F &
                        out$ld_islet == F,
                        "pancreas-unique",
                        out$annotation)

out$annotation = ifelse(out$category_before_ld == "fetal-adult",
                        "fetal-adult",
                        out$annotation)

# if the eqtls are in a failed module, annotate as "module_failed"
out$annotation = ifelse(out$module_pass == F & out$eqtl_type == "combinatorial", "module_failed", out$annotation)

# update module_pass column to be "NA" for singletons
out$module_pass = ifelse(out$eqtl_type == "combinatorial", out$module_pass, NA)

In [1006]:
table(out$annotation)


      adult-shared          ambiguous        fetal-adult        fetal-islet 
              2221              12098               3841                555 
    fetal-pancreas ipsc_ppc singleton    ipsc_ppc-unique    islet singleton 
               961               1518                302               2225 
      islet-unique      module_failed pancreas singleton    pancreas-unique 
               548                330               2358                427 

In [1012]:
table(out$eqtl_type)


combinatorial     singleton 
        12755         14629 

In [1015]:
out = out %>% dplyr::rename(category_after_ld = annotation)

In [1027]:
# annotate modules as ambiguous if at least one is in LD
mod_amb = out %>% filter(category_after_ld == "ambiguous" & eqtl_type == "combinatorial") %>% select(module_id, eqtl_type) %>% distinct()
out$category_after_ld = ifelse(out$module_id %in% mod_amb$module_id, "ambiguous", out$category_after_ld)

In [1028]:
fwrite(out, "reviews/tables/Table_S11_eQTL_Annotations_Final.txt", row.names = F, sep = "\t")

In [1030]:
table(out$category_after_ld, out$eqtl_type)

                    
                     combinatorial singleton
  adult-shared                2078         0
  ambiguous                   3981      8528
  fetal-adult                 3841         0
  fetal-islet                  494         0
  fetal-pancreas               839         0
  ipsc_ppc singleton             0      1518
  ipsc_ppc-unique              287         0
  islet singleton                0      2225
  islet-unique                 514         0
  module_failed                330         0
  pancreas singleton             0      2358
  pancreas-unique              391         0

In [1029]:
table(out$eqtl_phenotype, out$tissue)

                      
                       ipsc-ppc islet pancreas
  alternative_splicing     3959  4939     2077
  gene_expression          4149  3948     8312